In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings as wn
import sklearn.preprocessing as skpre
import category_encoders as ce

wn.simplefilter( "ignore" )

# Preparamos los Sets de Training

In [2]:
loc_ts = r"D:\FacundoTorraca\Documents\TP2_Machine_Learning\Training Sets"

In [3]:
ins_18_20 = pd.read_csv( loc_ts + "\\ins_18_20.csv" ); 
ins_21_23 = pd.read_csv( loc_ts + "\\ins_21_23.csv" ); 
ins_24_26 = pd.read_csv( loc_ts + "\\ins_24_26.csv" ); 

In [4]:
evt_18_20 = pd.read_csv( loc_ts + "\\evt_18_20.csv" );
evt_21_23 = pd.read_csv( loc_ts + "\\evt_21_23.csv" );
evt_24_26 = pd.read_csv( loc_ts + "\\evt_24_26.csv" );

In [5]:
auc_18_20 = pd.read_csv( loc_ts + "\\auc_18_20.csv" ); 
auc_21_23 = pd.read_csv( loc_ts + "\\auc_21_23.csv" );
auc_24_26 = pd.read_csv( loc_ts + "\\auc_24_26.csv" );

In [6]:
clk_18_20 = pd.read_csv( loc_ts + "\\clk_18_20.csv" );
clk_21_23 = pd.read_csv( loc_ts + "\\clk_21_23.csv" );
clk_24_26 = pd.read_csv( loc_ts + "\\clk_24_26.csv" );

# Genero los Sets con el primer install

In [7]:
ins_18_20["date"] =  pd.to_datetime( ins_18_20["date"] ); ins_18_20["_sc"] = ins_18_20["date"] - dt.datetime( year = 2019, month = 4, day = 18 ); ins_18_20["_sc"] = ins_18_20["_sc"].dt.total_seconds();
ins_21_23["date"] =  pd.to_datetime( ins_21_23["date"] ); ins_21_23["_sc"] = ins_21_23["date"] - dt.datetime( year = 2019, month = 4, day = 21 ); ins_21_23["_sc"] = ins_21_23["_sc"].dt.total_seconds(); 
ins_24_26["date"] =  pd.to_datetime( ins_24_26["date"] ); ins_24_26["_sc"] = ins_24_26["date"] - dt.datetime( year = 2019, month = 4, day = 24 ); ins_24_26["_sc"] = ins_24_26["_sc"].dt.total_seconds(); 

In [8]:
ins_18_20_first_ins = ins_18_20.sort_values( by = ["ref_hash","_sc"], ascending = True ).drop_duplicates( subset = ["ref_hash"], keep = "first" )
ins_21_23_first_ins = ins_21_23.sort_values( by = ["ref_hash","_sc"], ascending = True ).drop_duplicates( subset = ["ref_hash"], keep = "first" )
ins_24_26_first_ins = ins_24_26.sort_values( by = ["ref_hash","_sc"], ascending = True ).drop_duplicates( subset = ["ref_hash"], keep = "first" )

# Sacamos los Features

In [9]:
loc_ftr = r"D:\FacundoTorraca\Documents\TP2_Machine_Learning\Features\ftr_ins"

In [10]:
rh_18_20 = ins_18_20_first_ins[ ["ref_hash"] ] #RH unicos que coinciden con los datos que usamos para entrenar
rh_21_23 = ins_21_23_first_ins[ ["ref_hash"] ] #RH unicos que coinciden con los datos que queremos predecir

tg_18_20 = ins_18_20_first_ins[ ["_sc"] ]
tg_21_23 = ins_21_23_first_ins[ ["_sc"] ]

## <span style="color:green"> **Hora de la conversion** </span> 

Le aplicamos la transformacion sin( (hora * pi)/24 ) para agregarle periodicidad. Por ejemplo que las 00 hs tengan un valor similar a 23

In [897]:
hour_18_20 = rh_18_20.copy()
hour_21_23 = rh_21_23.copy()

In [ ]:
hour_18_20["sin_hour"] = pd.to_datetime( ins_18_20_first_ins["date"] ).dt.hour; hour_18_20["sin_hour"] = hour_18_20["sin_hour"].apply( lambda x: np.sin( (x *np.pi)/24 ) )
hour_21_23["sin_hour"] = pd.to_datetime( ins_21_23_first_ins["date"] ).dt.hour; hour_21_23["sin_hour"] = hour_21_23["sin_hour"].apply( lambda x: np.sin( (x *np.pi)/24 ) )

In [ ]:
hour_18_20.to_csv( loc_ftr + "\\sin_hour_18_20.csv", index = False )
hour_21_23.to_csv( loc_ftr + "\\sin_hour_21_23.csv", index = False )

## <span style="color:green"> **Cantidad de instalaciones por dispositivo en la ventana previa a la conversion** </span> 

In [898]:
cant_ins_18_20 = rh_18_20.copy()
cant_ins_21_23 = rh_21_23.copy()

In [50]:
cant_ins_18_20 = cant_ins_18_20.merge( ins_18_20["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "cant_ins", "index":"ref_hash"} ), how = "left", on = "ref_hash" )
cant_ins_21_23 = cant_ins_21_23.merge( ins_21_23["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "cant_ins", "index":"ref_hash"} ), how = "left", on = "ref_hash" )

In [51]:
cant_ins_18_20.to_csv( loc_ftr + "\\cant_ins_18_20.csv", index = False )
cant_ins_21_23.to_csv( loc_ftr + "\\cant_ins_21_23.csv", index = False )

## <span style="color:green"> **Cantidad de Eventos por dispositivo en la ventana previa a la conversion** </span>

In [29]:
cant_evt_18_20 = rh_18_20
cant_evt_21_23 = rh_21_23

In [30]:
cant_evt_18_20 = cant_evt_18_20.merge( evt_18_20["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "cant_evt", "index":"ref_hash"} ), how = "left", on = "ref_hash" )
cant_evt_21_23 = cant_evt_21_23.merge( evt_21_23["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "cant_evt", "index":"ref_hash"} ), how = "left", on = "ref_hash" )

In [32]:
cant_evt_18_20.fillna( 0, inplace = True )
cant_evt_21_23.fillna( 0, inplace = True )

In [33]:
cant_evt_18_20.to_csv( loc_ftr + "\\cant_evt_18_20.csv", index = False )
cant_evt_21_23.to_csv( loc_ftr + "\\cant_evt_21_23.csv", index = False )

## <span style="color:green"> **Cantidad de Subastas por dispositivo en la ventana previa a la conversion** </span>

In [115]:
cant_auc_18_20 = rh_18_20.copy()
cant_auc_21_23 = rh_21_23.copy()

In [116]:
cant_auc_18_20 = cant_auc_18_20.merge( auc_18_20["device_id"].value_counts().to_frame().reset_index().rename( columns = {"device_id": "cant_auc", "index":"ref_hash"} ), how = "left", on = "ref_hash" )
cant_auc_21_23 = cant_auc_21_23.merge( auc_21_23["device_id"].value_counts().to_frame().reset_index().rename( columns = {"device_id": "cant_auc", "index":"ref_hash"} ), how = "left", on = "ref_hash" )

In [117]:
cant_auc_18_20.fillna( 0, inplace = True )
cant_auc_21_23.fillna( 0, inplace = True )

In [118]:
cant_auc_18_20.to_csv( loc_ftr + "\\cant_auc_18_20.csv", index = False )
cant_auc_21_23.to_csv( loc_ftr + "\\cant_auc_21_23.csv", index = False )

## <span style="color:green"> **WiFi en la instalacion** </span>

#### <span style="color:orange"> **Mean Encoding** </span> (Usando el SC para el promedio) - <span style="color:red"> **No Mejora este tipo de codificacion** </span>

In [856]:
wifi_ins_18_20 = rh_18_20.copy()
wifi_ins_21_23 = rh_21_23.copy()

In [611]:
sc = ins_21_23_first_ins.loc[ ins_21_23_first_ins["ref_hash"].isin( rh_18_20["ref_hash"] ), ["ref_hash","_sc"] ]

wf_18_20_sc = ins_18_20_first_ins[ ["ref_hash","wifi"] ].merge( sc, how = "left", on = "ref_hash" ); 

X_wf = wf_18_20_sc[ ["wifi"] ].astype(str)
Y_wf = wf_18_20_sc[ ["_sc"] ]

In [612]:
te = ce.TargetEncoder(smoothing=True); te.fit( X_wf, Y_wf );

In [613]:
wifi_ins_18_20["wifi"] = te.transform( ins_18_20_first_ins[ ["wifi"] ].astype( str ) );
wifi_ins_21_23["wifi"] = te.transform( ins_21_23_first_ins[ ["wifi"] ].astype( str ) ); 

In [616]:
wifi_ins_18_20.to_csv( loc_ftr + "\\wifi_ins_18_20.csv", index = False )
wifi_ins_21_23.to_csv( loc_ftr + "\\wifi_ins_21_23.csv", index = False )

#### <span style="color:orange"> **OneHotEncoding** </span> 

In [894]:
wifi_ins_18_20 = rh_18_20.copy()
wifi_ins_21_23 = rh_21_23.copy()

In [623]:
wf_ohe_18_20 = pd.get_dummies( ins_18_20_first_ins[ ["wifi"] ] )
wf_ohe_21_23 = pd.get_dummies( ins_21_23_first_ins[ ["wifi"] ] )

In [624]:
wifi_ins_18_20 = pd.concat( [wifi_ins_18_20, wf_ohe_18_20], axis = 1 )
wifi_ins_21_23 = pd.concat( [wifi_ins_21_23, wf_ohe_21_23], axis = 1 )

In [625]:
wifi_ins_18_20.to_csv( loc_ftr + "\\wifi_ins_18_20.csv", index = False )
wifi_ins_21_23.to_csv( loc_ftr + "\\wifi_ins_21_23.csv", index = False )

## <span style="color:green"> **Aplicacion mas instalada por el usuario** </span>

#### <span style="color:orange"> **Mean Encoding** </span> (Usamos el promedio de la cantidad de veces que que es la app principal de algun dispositivo)

In [19]:
main_app_18_20 = rh_18_20.copy()
main_app_21_23 = rh_21_23.copy()

app_mas_ins_18_20 = ins_18_20.groupby( by = ["ref_hash","application_id"] ).agg( {"application_id":"count"} ).rename( columns = {"application_id":"cant_ins"} ).reset_index()
app_mas_ins_21_23 = ins_21_23.groupby( by = ["ref_hash","application_id"] ).agg( {"application_id":"count"} ).rename( columns = {"application_id":"cant_ins"} ).reset_index()

app_mas_ins_18_20 = app_mas_ins_18_20.sort_values( by = ["ref_hash", "cant_ins"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( app_mas_ins_18_20["cant_ins"] )
app_mas_ins_21_23 = app_mas_ins_21_23.sort_values( by = ["ref_hash", "cant_ins"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( app_mas_ins_21_23["cant_ins"] ) 

main_app_18_20 = main_app_18_20.merge( app_mas_ins_18_20, how = "left", on = "ref_hash" )
main_app_21_23 = main_app_21_23.merge( app_mas_ins_21_23, how = "left", on = "ref_hash" )

main_app_18_20["to_count"] = 1; main_app_18_20["application_id"] = main_app_18_20[["application_id", "to_count"]].groupby("application_id").transform( "sum" ) / len(main_app_18_20); del(main_app_18_20["to_count"])
main_app_21_23["to_count"] = 1; main_app_21_23["application_id"] = main_app_21_23[["application_id", "to_count"]].groupby("application_id").transform( "sum" ) / len(main_app_21_23); del(main_app_21_23["to_count"])

main_app_18_20.to_csv( loc_ftr + "\\main_app_18_20.csv", index = False )
main_app_21_23.to_csv( loc_ftr + "\\main_app_21_23.csv", index = False )

## <span style="color:green"> **Cantidad de instalaciones implicitas por dispositivo** </span> 

In [873]:
cins_imp_18_20 = rh_18_20.copy()
cins_imp_21_23 = rh_21_23.copy()

In [873]:
cins_imp_18_20["cant_imp"] = ins_18_20["implicit"].apply( lambda x: 1 if x else 0 ); cins_imp_18_20 = cins_imp_18_20.groupby( "ref_hash" ).agg( {"cant_imp":"sum"} ).reset_index()
cins_imp_21_23["cant_imp"] = ins_21_23["implicit"].apply( lambda x: 1 if x else 0 ); cins_imp_21_23 = cins_imp_21_23.groupby( "ref_hash" ).agg( {"cant_imp":"sum"} ).reset_index()

In [873]:
cins_imp_18_20.to_csv( loc_ftr + "\\cins_imp_18_20.csv", index = False )
cins_imp_21_23.to_csv( loc_ftr + "\\cins_imp_21_23.csv", index = False )

## <span style="color:green"> **Tipo de la primera instalacion (implicita/no implicita)** </span> 

#### <span style="color:orange"> **OneHotEncoding** </span> 

In [891]:
type_ins_18_20 = rh_18_20.copy()
type_ins_21_23 = rh_21_23.copy()

In [879]:
type_ins_ohe_18_20 = pd.get_dummies( ins_18_20_first_ins[ ["implicit"] ].astype( "category" ) )
type_ins_ohe_21_23 = pd.get_dummies( ins_21_23_first_ins[ ["implicit"] ].astype( "category" ) )

In [881]:
type_ins_18_20 = pd.concat( [type_ins_18_20, type_ins_ohe_18_20], axis = 1 )
type_ins_21_23 = pd.concat( [type_ins_21_23, type_ins_ohe_21_23], axis = 1 )

In [882]:
type_ins_18_20.to_csv( loc_ftr + "\\type_ins_18_20.csv", index = False )
type_ins_21_23.to_csv( loc_ftr + "\\type_ins_21_23.csv", index = False )

## <span style="color:green"> **Ref Type del dispositivo que realizo la instalacion** </span>

#### <span style="color:orange"> **Mean Encoding** </span> (Usamos el promedio de la cantidad de instalaciones con ese ref_type en la ventana)

In [1016]:
ref_type_18_20 = rh_18_20.copy()
ref_type_21_23 = rh_21_23.copy()

In [1017]:
rt_18_20_mean = ins_18_20[ ["ref_type"] ].groupby( "ref_type" ).agg( {"ref_type":"count"} ).rename( columns = {"ref_type":"cant_ins"} ).reset_index()
rt_21_23_mean = ins_21_23[ ["ref_type"] ].groupby( "ref_type" ).agg( {"ref_type":"count"} ).rename( columns = {"ref_type":"cant_ins"} ).reset_index()

In [1018]:
rt_18_20_mean["ref_type_mean"] = rt_18_20_mean["cant_ins"] / len(ins_18_20); del( rt_18_20_mean["cant_ins"] )
rt_21_23_mean["ref_type_mean"] = rt_21_23_mean["cant_ins"] / len(ins_21_23); del( rt_21_23_mean["cant_ins"] )

In [1019]:
ref_type_18_20["ref_type"] = ins_18_20_first_ins["ref_type"]; ref_type_18_20 = ref_type_18_20.merge( rt_18_20_mean, how = "left", on = "ref_type" ); del( ref_type_18_20["ref_type"] )
ref_type_21_23["ref_type"] = ins_21_23_first_ins["ref_type"]; ref_type_21_23 = ref_type_21_23.merge( rt_21_23_mean, how = "left", on = "ref_type" ); del( ref_type_21_23["ref_type"] )

In [1021]:
ref_type_18_20.to_csv( loc_ftr + "\\ref_type_18_20.csv", index = False )
ref_type_21_23.to_csv( loc_ftr + "\\ref_type_21_23.csv", index = False )

## <span style="color:green"> **Cantidad de instalaciones atribuidas por dispositivo** </span> 

In [79]:
cins_atr_18_20 = rh_18_20.copy()
cins_atr_21_23 = rh_21_23.copy()

In [79]:
cins_atr_18_20["cant_atr"] = ins_18_20["attributed"].apply( lambda x: 1 if x else 0 ); cins_atr_18_20 = cins_atr_18_20.groupby( "ref_hash" ).agg( {"cant_atr":"sum"} ).reset_index()
cins_atr_21_23["cant_atr"] = ins_21_23["attributed"].apply( lambda x: 1 if x else 0 ); cins_atr_21_23 = cins_atr_21_23.groupby( "ref_hash" ).agg( {"cant_atr":"sum"} ).reset_index()

In [79]:
cins_atr_18_20.to_csv( loc_ftr + "\\cins_atr_18_20.csv", index = False )
cins_atr_21_23.to_csv( loc_ftr + "\\cins_atr_21_23.csv", index = False )

## <span style="color:green"> **IP del dispositivo al realizar la primera instalacion** </span> 

In [108]:
ip_encod_18_20 = rh_18_20.copy()
ip_encod_21_23 = rh_21_23.copy()

In [109]:
ip_encod_18_20["ip_address"] = ins_18_20_first_ins["ip_address"];
ip_encod_21_23["ip_address"] = ins_21_23_first_ins["ip_address"];

In [110]:
appces_18_20_ins = ins_18_20["ip_address"].value_counts().reset_index().rename( columns = {"index":"ip_address", "ip_address":"appearances"} )
appces_21_23_ins = ins_21_23["ip_address"].value_counts().reset_index().rename( columns = {"index":"ip_address", "ip_address":"appearances"} )

In [111]:
appces_18_20_ins["ip_address_mean"] = appces_18_20_ins["appearances"] / len( ins_18_20 ); appces_18_20_ins.drop( "appearances", axis = 1, inplace = True )
appces_21_23_ins["ip_address_mean"] = appces_21_23_ins["appearances"] / len( ins_21_23 ); appces_21_23_ins.drop( "appearances", axis = 1, inplace = True )

In [112]:
ip_encod_18_20 = ip_encod_18_20.merge( appces_18_20_ins, how = "left", on = "ip_address" ).drop( "ip_address", axis = 1 )
ip_encod_21_23 = ip_encod_21_23.merge( appces_21_23_ins, how = "left", on = "ip_address" ).drop( "ip_address", axis = 1 )

In [114]:
ip_encod_18_20.to_csv( loc_ftr + "\\ip_encod_18_20.csv", index = False )
ip_encod_21_23.to_csv( loc_ftr + "\\ip_encod_21_23.csv", index = False )

## <span style="color:green"> **Cantidad de eventos atribuidas por dispositivo** </span> 

In [82]:
cevt_atr_18_20 = rh_18_20.copy()
cevt_atr_21_23 = rh_21_23.copy()

In [83]:
cant_atr_evt_18_20 = evt_18_20[["ref_hash", "attributed"]]; cant_atr_evt_18_20["attributed"] = cant_atr_evt_18_20["attributed"].apply( lambda x: 1 if x else 0 );
cant_atr_evt_21_23 = evt_21_23[["ref_hash", "attributed"]]; cant_atr_evt_21_23["attributed"] = cant_atr_evt_21_23["attributed"].apply( lambda x: 1 if x else 0 );

In [84]:
cant_atr_evt_18_20 = cant_atr_evt_18_20.groupby( "ref_hash" ).agg( "sum" )
cant_atr_evt_21_23 = cant_atr_evt_21_23.groupby( "ref_hash" ).agg( "sum" )

In [85]:
cevt_atr_18_20 = cevt_atr_18_20.merge( cant_atr_evt_18_20, how = "left", on = "ref_hash" ).fillna( 0 )
cevt_atr_21_23 = cevt_atr_21_23.merge( cant_atr_evt_21_23, how = "left", on = "ref_hash" ).fillna( 0 )

In [86]:
cevt_atr_18_20.to_csv( loc_ftr + "\\cevt_atr_18_20.csv", index = False )
cevt_atr_21_23.to_csv( loc_ftr + "\\cevt_atr_21_23.csv", index = False )

## <span style="color:green"> **Tipo de evento mas realizado por el dispositivo** </span> 

#### <span style="color:orange"> **Mean Encoding** </span> (Lo codificamos haciendo la cantidad de eventos que se realizaron de ese tipo sobre el total de eventos realizados)

In [144]:
kind_evt_18_20 = rh_18_20.copy()
kind_evt_21_23 = rh_21_23.copy()

In [145]:
main_kind_evt_18_20 = evt_18_20.groupby( by = ["ref_hash","kind"] ).agg( {"kind":"count"} ).rename( columns = {"kind":"cant_evt"} ).reset_index()
main_kind_evt_21_23 = evt_21_23.groupby( by = ["ref_hash","kind"] ).agg( {"kind":"count"} ).rename( columns = {"kind":"cant_evt"} ).reset_index()

In [146]:
main_kind_evt_18_20 = main_kind_evt_18_20.sort_values( by = ["ref_hash", "cant_evt"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( main_kind_evt_18_20["cant_evt"] )
main_kind_evt_21_23 = main_kind_evt_21_23.sort_values( by = ["ref_hash", "cant_evt"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( main_kind_evt_21_23["cant_evt"] ) 

In [147]:
kind_evt_18_20 = kind_evt_18_20.merge( main_kind_evt_18_20, how = "left", on = "ref_hash" )
kind_evt_21_23 = kind_evt_21_23.merge( main_kind_evt_21_23, how = "left", on = "ref_hash" )

In [148]:
cant_kind_evt_18_20 = evt_18_20[["ref_hash","kind"]].groupby( "kind" ).agg("count").reset_index().rename( columns = {"ref_hash":"cant_kind"} )
cant_kind_evt_21_23 = evt_21_23[["ref_hash","kind"]].groupby( "kind" ).agg("count").reset_index().rename( columns = {"ref_hash":"cant_kind"} )

In [149]:
kind_evt_18_20 = kind_evt_18_20.merge( cant_kind_evt_18_20, how = "left", on = "kind" ).drop( "kind", axis = 1 )
kind_evt_21_23 = kind_evt_21_23.merge( cant_kind_evt_21_23, how = "left", on = "kind" ).drop( "kind", axis = 1 )

In [150]:
kind_evt_18_20["kind_mean"] = kind_evt_18_20["cant_kind"] / len(evt_18_20); kind_evt_18_20.fillna( kind_evt_18_20["kind_mean"].isnull().sum() / len(evt_18_20), inplace = True ); del(kind_evt_18_20["cant_kind"])
kind_evt_21_23["kind_mean"] = kind_evt_21_23["cant_kind"] / len(evt_21_23); kind_evt_21_23.fillna( kind_evt_21_23["kind_mean"].isnull().sum() / len(evt_21_23), inplace = True ); del(kind_evt_21_23["cant_kind"])

In [152]:
kind_evt_18_20.to_csv( loc_ftr + "\\kind_evt_18_20.csv", index = False )
kind_evt_21_23.to_csv( loc_ftr + "\\kind_evt_21_23.csv", index = False )

## <span style="color:green"> **Tiempo hasta la primer auction en ese ventana** </span> 

Le asignamos cuanto tiempo, en la ventana del 18-20, tardo en realizar su primer auction

In [12]:
frst_auc_18_20 = rh_18_20.copy()
frst_auc_21_23 = rh_21_23.copy()

In [13]:
first_auctions_rh_ins_18_20 = auc_18_20[ ["device_id","date"] ].sort_values( "date" ).drop_duplicates( subset = "device_id", keep = "first" ).rename( columns = {"device_id":"ref_hash"} )
first_auctions_rh_ins_21_23 = auc_21_23[ ["device_id","date"] ].sort_values( "date" ).drop_duplicates( subset = "device_id", keep = "first" ).rename( columns = {"device_id":"ref_hash"} )

In [19]:
first_auctions_rh_ins_18_20["time_to_frt_auc"] = ( pd.to_datetime( first_auctions_rh_ins_18_20["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
first_auctions_rh_ins_21_23["time_to_frt_auc"] = ( pd.to_datetime( first_auctions_rh_ins_21_23["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

In [25]:
first_auctions_rh_ins_18_20.drop( "date", axis = 1, inplace = True )
first_auctions_rh_ins_21_23.drop( "date", axis = 1, inplace = True )

In [26]:
frst_auc_18_20 = frst_auc_18_20.merge( first_auctions_rh_ins_18_20, how = "left", on = "ref_hash" )
frst_auc_21_23 = frst_auc_21_23.merge( first_auctions_rh_ins_21_23, how = "left", on = "ref_hash" )

In [29]:
#Los que tienen NaN es que nunca convirtieron. Los marcamos con el tiempo maximo
frst_auc_18_20.fillna( 3 * 24 * 3600, inplace = True )
frst_auc_21_23.fillna( 3 * 24 * 3600, inplace = True )

In [30]:
frst_auc_18_20.to_csv( loc_ftr + "\\frst_auc_18_20.csv", index = False )
frst_auc_21_23.to_csv( loc_ftr + "\\frst_auc_21_23.csv", index = False )

## <span style="color:green"> **Tiempo hasta el primer evento en ese ventana** </span> 

Le asignamos cuanto tiempo, en la ventana del 18-20, tardo en realizar su primer evento

In [46]:
frst_evt_18_20 = rh_18_20.copy()
frst_evt_21_23 = rh_21_23.copy()

In [46]:
first_event_rh_ins_18_20 = evt_18_20[ ["ref_hash","date"] ].sort_values( "date" ).drop_duplicates( subset = "ref_hash", keep = "first" )
first_event_rh_ins_21_23 = evt_21_23[ ["ref_hash","date"] ].sort_values( "date" ).drop_duplicates( subset = "ref_hash", keep = "first" )

In [46]:
first_event_rh_ins_18_20["time_to_frt_evt"] = ( pd.to_datetime( first_event_rh_ins_18_20["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
first_event_rh_ins_21_23["time_to_frt_evt"] = ( pd.to_datetime( first_event_rh_ins_21_23["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

In [46]:
first_event_rh_ins_18_20.drop( "date", axis = 1, inplace = True )
first_event_rh_ins_21_23.drop( "date", axis = 1, inplace = True )

In [46]:
frst_evt_18_20 = frst_evt_18_20.merge( first_event_rh_ins_18_20, how = "left", on = "ref_hash" )
frst_evt_21_23 = frst_evt_21_23.merge( first_event_rh_ins_21_23, how = "left", on = "ref_hash" )

In [46]:
#Los que tienen NaN es que nunca convirtieron. Los marcamos con el tiempo maximo
frst_evt_18_20.fillna( 3 * 24 * 3600, inplace = True )
frst_evt_21_23.fillna( 3 * 24 * 3600, inplace = True )

In [46]:
frst_evt_18_20.to_csv( loc_ftr + "\\frst_evt_18_20.csv", index = False )
frst_evt_21_23.to_csv( loc_ftr + "\\frst_evt_21_23.csv", index = False )

## <span style="color:green"> **Horario del primer evento en ese ventana** </span> 

Al agregarle la transformacion senoidal para agregarle perdiodicidad a la hora, la prediccion del algoritmo fue peor por lo que decidimos no agregarla

In [93]:
hr_f_evt_18_20 = rh_18_20.copy()
hr_f_evt_21_23 = rh_21_23.copy()

In [94]:
first_evt_hour_18_20 = evt_18_20[ ["ref_hash","date"] ].sort_values( "date" ).drop_duplicates( subset = "ref_hash", keep = "first" )
first_evt_hour_21_23 = evt_21_23[ ["ref_hash","date"] ].sort_values( "date" ).drop_duplicates( subset = "ref_hash", keep = "first" )

In [95]:
first_evt_hour_18_20["time_to_frt_evt"] = ( pd.to_datetime( first_evt_hour_18_20["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
first_evt_hour_21_23["time_to_frt_evt"] = ( pd.to_datetime( first_evt_hour_21_23["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

In [96]:
first_evt_hour_18_20["hour_frt_evt"] = pd.to_datetime( first_evt_hour_18_20["date"] ).dt.hour
first_evt_hour_21_23["hour_frt_evt"] = pd.to_datetime( first_evt_hour_21_23["date"] ).dt.hour

In [97]:
first_evt_hour_18_20.drop( ["date", "time_to_frt_evt"], axis = 1, inplace = True )
first_evt_hour_21_23.drop( ["date", "time_to_frt_evt"], axis = 1, inplace = True )

In [98]:
hr_f_evt_18_20 = hr_f_evt_18_20.merge( first_evt_hour_18_20, how = "left", on = "ref_hash" )
hr_f_evt_21_23 = hr_f_evt_21_23.merge( first_evt_hour_21_23, how = "left", on = "ref_hash" )

In [99]:
#hr_f_evt_18_20["hour_frt_evt"] = hr_f_evt_18_20["hour_frt_evt"].apply( lambda x: np.sin( (x *np.pi)/24 ) )
#hr_f_evt_21_23["hour_frt_evt"] = hr_f_evt_21_23["hour_frt_evt"].apply( lambda x: np.sin( (x *np.pi)/24 ) )

In [100]:
hr_f_evt_18_20.to_csv( loc_ftr + "\\hr_f_evt_18_20.csv", index = False )
hr_f_evt_21_23.to_csv( loc_ftr + "\\hr_f_evt_21_23.csv", index = False )

Si bien hay varios valores NaN en el feature, lo decidimos probar de igual manera, ya que los algoritmos de boosting aceptan valores NaNs.

* https://datascience.stackexchange.com/questions/15305/how-does-xgboost-learn-what-are-the-inputs-for-missing-values/15306#15306
* The procedure is described in [their paper, section 3.4: Sparsity aware split-finding](https://arxiv.org/pdf/1603.02754v3.pdf).


## <span style="color:green"> **Tiempo hasta la ultima auction en esa ventana** </span> 

In [11]:
last_auc_18_20 = rh_18_20.copy()
last_auc_21_23 = rh_21_23.copy()

last_auctions_rh_ins_18_20 = auc_18_20[ ["device_id","date"] ].sort_values( "date", ascending = False ).drop_duplicates( subset = "device_id", keep = "first" ).rename( columns = {"device_id":"ref_hash"} )
last_auctions_rh_ins_21_23 = auc_21_23[ ["device_id","date"] ].sort_values( "date", ascending = False ).drop_duplicates( subset = "device_id", keep = "first" ).rename( columns = {"device_id":"ref_hash"} )

last_auctions_rh_ins_18_20["time_to_lst_auc"] = ( pd.to_datetime( last_auctions_rh_ins_18_20["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
last_auctions_rh_ins_21_23["time_to_lst_auc"] = ( pd.to_datetime( last_auctions_rh_ins_21_23["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

last_auctions_rh_ins_18_20.drop( "date", axis = 1, inplace = True )
last_auctions_rh_ins_21_23.drop( "date", axis = 1, inplace = True )

last_auc_18_20 = last_auc_18_20.merge( last_auctions_rh_ins_18_20, how = "left", on = "ref_hash" )
last_auc_21_23 = last_auc_21_23.merge( last_auctions_rh_ins_21_23, how = "left", on = "ref_hash" )

#Los que tienen NaN es que nunca convirtieron. Los marcamos con el tiempo maximo
last_auc_18_20.fillna( 3 * 24 * 3600, inplace = True )
last_auc_21_23.fillna( 3 * 24 * 3600, inplace = True )

last_auc_18_20.to_csv( loc_ftr + "\\last_auc_18_20.csv", index = False )
last_auc_21_23.to_csv( loc_ftr + "\\last_auc_21_23.csv", index = False )

## <span style="color:green"> **Tiempo hasta la ultimo evento en esa ventana** </span> 

In [12]:
last_evt_18_20 = rh_18_20.copy()
last_evt_21_23 = rh_21_23.copy()

last_event_rh_ins_18_20 = evt_18_20[ ["ref_hash","date"] ].sort_values( "date", ascending = False ).drop_duplicates( subset = "ref_hash", keep = "first" )
last_event_rh_ins_21_23 = evt_21_23[ ["ref_hash","date"] ].sort_values( "date", ascending = False ).drop_duplicates( subset = "ref_hash", keep = "first" )

last_event_rh_ins_18_20["time_to_lst_evt"] = ( pd.to_datetime( last_event_rh_ins_18_20["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
last_event_rh_ins_21_23["time_to_lst_evt"] = ( pd.to_datetime( last_event_rh_ins_21_23["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

last_event_rh_ins_18_20.drop( "date", axis = 1, inplace = True )
last_event_rh_ins_21_23.drop( "date", axis = 1, inplace = True )

last_evt_18_20 = last_evt_18_20.merge( last_event_rh_ins_18_20, how = "left", on = "ref_hash" )
last_evt_21_23 = last_evt_21_23.merge( last_event_rh_ins_21_23, how = "left", on = "ref_hash" )

#Los que tienen NaN es que nunca convirtieron. Los marcamos con el tiempo maximo
last_evt_18_20.fillna( 3 * 24 * 3600, inplace = True )
last_evt_21_23.fillna( 3 * 24 * 3600, inplace = True )

last_evt_18_20.to_csv( loc_ftr + "\\last_evt_18_20.csv", index = False )
last_evt_21_23.to_csv( loc_ftr + "\\last_evt_21_23.csv", index = False )

## <span style="color:green"> **Participo en mas de una subasta** </span> 

In [14]:
cant_auc_18_20 = rh_18_20.copy()
cant_auc_21_23 = rh_21_23.copy()

cant_auc_18_20 = cant_auc_18_20.merge( auc_18_20["device_id"].value_counts().to_frame().reset_index().rename( columns = {"device_id": "cant_auc", "index":"ref_hash"} ), how = "left", on = "ref_hash" )
cant_auc_21_23 = cant_auc_21_23.merge( auc_21_23["device_id"].value_counts().to_frame().reset_index().rename( columns = {"device_id": "cant_auc", "index":"ref_hash"} ), how = "left", on = "ref_hash" )

cant_auc_18_20.fillna( 0, inplace = True )
cant_auc_21_23.fillna( 0, inplace = True )

cant_auc_18_20['cant_auc'] = (cant_auc_18_20['cant_auc'] > 1).astype('int8')
cant_auc_21_23['cant_auc'] = (cant_auc_21_23['cant_auc'] > 1).astype('int8')

cant_auc_18_20.to_csv( loc_ftr + "\\mas1_auc_18_20.csv", index = False )
cant_auc_21_23.to_csv( loc_ftr + "\\mas1_auc_21_23.csv", index = False )

## <span style="color:green"> **Recibio subasta entre 21 hs y 3 hs (Noche)** </span> 

In [16]:
auc_18_20['auc_21_3'] = ( pd.to_datetime( auc_18_20['date'] ).dt.hour < 4) | ( pd.to_datetime( auc_18_20['date'] ).dt.hour > 20)
auc_21_23['auc_21_3'] = ( pd.to_datetime( auc_21_23['date'] ).dt.hour < 4) | ( pd.to_datetime( auc_21_23['date'] ).dt.hour > 20)

auc_night_18_20 = rh_18_20.copy()
auc_night_21_23 = rh_21_23.copy()

hour_mode_18_20 = auc_18_20.groupby('device_id').agg({'auc_21_3':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )
hour_mode_21_23 = auc_21_23.groupby('device_id').agg({'auc_21_3':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )

auc_night_18_20 = auc_night_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
auc_night_21_23 = auc_night_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

auc_night_18_20["auc_21_3"] =  (auc_night_18_20["auc_21_3"] > 1).astype('int8')
auc_night_21_23["auc_21_3"] =  (auc_night_21_23["auc_21_3"] > 1).astype('int8')

auc_night_18_20.to_csv( loc_ftr + "\\auc_night_18_20.csv", index = False )
auc_night_21_23.to_csv( loc_ftr + "\\auc_night_21_23.csv", index = False )

del auc_18_20['auc_21_3']
del auc_21_23['auc_21_3']

## <span style="color:green"> **Recibio clicks entre 16 hs y 20 hs (Tarde)** </span> 

In [35]:
clk_18_20['clk_16_20'] = ( pd.to_datetime( clk_18_20['date'] ).dt.hour > 15) & ( pd.to_datetime( clk_18_20['date'] ).dt.hour < 21) 
clk_21_23['clk_16_20'] = ( pd.to_datetime( clk_21_23['date'] ).dt.hour > 15) & ( pd.to_datetime( clk_21_23['date'] ).dt.hour < 21)

clk_after_18_20 = rh_18_20.copy()
clk_after_21_23 = rh_21_23.copy()

hour_mode_18_20 = clk_18_20.groupby('ref_hash').agg({'clk_16_20':'sum'}).reset_index()
hour_mode_21_23 = clk_21_23.groupby('ref_hash').agg({'clk_16_20':'sum'}).reset_index()

clk_after_18_20 = clk_after_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
clk_after_21_23 = clk_after_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

clk_after_18_20["clk_16_20"] = (clk_after_18_20["clk_16_20"] > 1).astype('int8')
clk_after_21_23["clk_16_20"] = (clk_after_21_23["clk_16_20"] > 1).astype('int8')

clk_after_18_20.to_csv( loc_ftr + "\\clk_after_18_20.csv", index = False )
clk_after_21_23.to_csv( loc_ftr + "\\clk_after_21_23.csv", index = False )

del clk_18_20['clk_16_20']
del clk_21_23['clk_16_20']

## <span style="color:green"> **Recibio clikcs entre 21 hs y 3 hs (Noche)** </span> 

In [36]:
clk_18_20['clk_21_3'] = ( pd.to_datetime( clk_18_20['date'] ).dt.hour < 4) | ( pd.to_datetime( clk_18_20['date'] ).dt.hour > 20)
clk_21_23['clk_21_3'] = ( pd.to_datetime( clk_21_23['date'] ).dt.hour < 4) | ( pd.to_datetime( clk_21_23['date'] ).dt.hour > 20)

clk_night_18_20 = rh_18_20.copy()
clk_night_21_23 = rh_21_23.copy()

hour_mode_18_20 = clk_18_20.groupby('ref_hash').agg({'clk_21_3':'sum'}).reset_index()
hour_mode_21_23 = clk_21_23.groupby('ref_hash').agg({'clk_21_3':'sum'}).reset_index()

clk_night_18_20 = clk_night_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
clk_night_21_23 = clk_night_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

clk_night_18_20["clk_21_3"] =  (clk_night_18_20["clk_21_3"] > 1).astype('int8')
clk_night_21_23["clk_21_3"] =  (clk_night_21_23["clk_21_3"] > 1).astype('int8')

clk_night_18_20.to_csv( loc_ftr + "\\clk_night_18_20.csv", index = False )
clk_night_21_23.to_csv( loc_ftr + "\\clk_night_21_23.csv", index = False )

del clk_18_20['clk_21_3']
del clk_21_23['clk_21_3']

## <span style="color:green"> **Recibio clicks entre 11 hs y 15 hs (Medio dia)** </span> 

In [38]:
clk_18_20['clk_11_15'] = ( pd.to_datetime( clk_18_20['date'] ).dt.hour > 10) & ( pd.to_datetime( clk_18_20['date'] ).dt.hour < 16) 
clk_21_23['clk_11_15'] = ( pd.to_datetime( clk_21_23['date'] ).dt.hour > 10) & ( pd.to_datetime( clk_21_23['date'] ).dt.hour < 16)

clk_midday_18_20 = rh_18_20.copy()
clk_midday_21_23 = rh_21_23.copy()

hour_mode_18_20 = clk_18_20.groupby('ref_hash').agg({'clk_11_15':'sum'}).reset_index()
hour_mode_21_23 = clk_21_23.groupby('ref_hash').agg({'clk_11_15':'sum'}).reset_index()

clk_midday_18_20 = clk_midday_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
clk_midday_21_23 = clk_midday_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

clk_midday_18_20["clk_11_15"] =  (clk_midday_18_20["clk_11_15"] > 1).astype('int8')
clk_midday_21_23["clk_11_15"] =  (clk_midday_21_23["clk_11_15"] > 1).astype('int8')

clk_midday_18_20.to_csv( loc_ftr + "\\clk_midday_18_20.csv", index = False )
clk_midday_21_23.to_csv( loc_ftr + "\\clk_midday_21_23.csv", index = False )

del clk_18_20['clk_11_15']
del clk_21_23['clk_11_15']

## <span style="color:green"> **Recibio clicks entre 4 hs y 10 hs (Mañana)** </span> 

In [39]:
clk_18_20['clk_4_10'] = ( pd.to_datetime( clk_18_20['date'] ).dt.hour > 3) & ( pd.to_datetime( clk_18_20['date'] ).dt.hour < 11) 
clk_21_23['clk_4_10'] = ( pd.to_datetime( clk_21_23['date'] ).dt.hour > 3) & ( pd.to_datetime( clk_21_23['date'] ).dt.hour < 11)

clk_morn_18_20 = rh_18_20.copy()
clk_morn_21_23 = rh_21_23.copy()

hour_mode_18_20 = clk_18_20.groupby('ref_hash').agg({'clk_4_10':'sum'}).reset_index()
hour_mode_21_23 = clk_21_23.groupby('ref_hash').agg({'clk_4_10':'sum'}).reset_index()

clk_morn_18_20 = clk_morn_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
clk_morn_21_23 = clk_morn_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

clk_morn_18_20["ins_4_10"] =  (clk_morn_18_20["clk_4_10"] > 1).astype('int8')
clk_morn_21_23["ins_4_10"] =  (clk_morn_21_23["clk_4_10"] > 1).astype('int8')

clk_morn_18_20.to_csv( loc_ftr + "\\clk_morn_18_20.csv", index = False )
clk_morn_21_23.to_csv( loc_ftr + "\\clk_morn_21_23.csv", index = False )

del clk_18_20['clk_4_10']
del clk_21_23['clk_4_10']